In [ ]:
# !pip install pycountry_convert

# !pip install reverse_geocode

In [275]:
from datetime import datetime
import json
from xml.etree import ElementTree as ET
import reverse_geocode as rg
import pycountry_convert as pc


In [ ]:
def is_already_in_table(diz_row, table, generare_ID):
    if not generare_ID: # ho già ID nel Police.csv, non dobbiamo crearlo noi #è il caso ad esempio di Date 
        # check se l'ID è già presente in table
        id_to_check = None
        for k, v in diz_row.items():  
            if k.endswith('_fk') or k.endswith('_id'):
                id_to_check = v  
                break 
        if id_to_check in table.keys():
            return id_to_check
        return None # non è ancora presente 
     
    else: # l'ID non è già dato ma va generato, quindi non posso fare un check su ID (non ce l'ho)
        valori_row = list(diz_row.values())
        for k, v in table.items():
            if valori_row == v:
                return int(k)
    return None


def add_record(diz_row, table, generare_ID):
    id_key = None

    for k, v in diz_row.items():
        if k.endswith('_fk') or k.endswith('_id'):
            id_key = k
            break

    if not table:
        if generare_ID:
            table[1] = list(diz_row.values())
            return 1
        else:
            if id_key is not None:
                id = diz_row[id_key]
                table[id] = {k: v for k, v in diz_row.items() if k != id_key}
                return id
            else:
                return None

    already_in = is_already_in_table(diz_row, table, generare_ID)
    if already_in is None:
        if generare_ID: # adesso devo creare ID incrementale
            pk = list(table.keys())[-1] + 1
            table[pk] = list(diz_row.values())
            return pk 
        else: 
            id = None
            for k, v in diz_row.items():  
                if k.endswith('_fk') or k.endswith('_id'):
                    id = v  
                    break
            table[id] =  [{k: v for k, v in diz_row.items() if k != id_key}]
            return id

    else: # already_in not None 

        return already_in


def data_into_tables(rows, tables):
    ids = []
    for table_name, row in zip(tables.keys(), rows):
        table, generate_ID = tables[table_name]  # Unpack the tuple

        if table_name == 'geography':
            geo_key = (row['latitude'], row['longitude'])
            if geo_key in table:
                geo_fk = table[geo_key]
            else:
                # Generate a new geo_id if not found
                new_geo_id = len(table) + 1
                table[geo_key] = new_geo_id
                geo_fk = new_geo_id
            ids_line = geo_fk  # Assign geo_fk to ids_line
        else:
            ids_line = add_record(row, table, generate_ID)
        ids.append(ids_line)
    return ids

In [ ]:
def compute_gravity(age, type, status, partecipant_info):
    
    ''' Crime gravity attribute is the main measure of the data warehouse. 
        You can compute its values using Eq. and the additional files .json
        Given an instance x, 
        crime gravity(x) = F1(x.partecipant age)∗F2(x.partecipant type)∗F3(x.partecipant status)'''
        

    partecipant_age = partecipant_info.get('participant_age_group', None)
    partecipant_type = partecipant_info.get('participant_type', None)
    partecipant_status = partecipant_info.get('participant_status', None)
    crime_gravity = age[partecipant_age] * type[partecipant_type] * status[partecipant_status]

    return crime_gravity

In [ ]:
def process_police(file_police, dimensional_tables, incident, custody, age, type, status): 
    with open(file_police, 'r') as police:
        header = None
        for line in police:
            if header is None:
                header = line.strip().split(',')
            else:
                row_values = line.strip().split(',')
                row_dict = {header[i]: row_values[i] for i in range(len(header))}

                partec_info = {
                    'participant_age_group': row_dict['participant_age_group'],
                    'participant_gender': row_dict['participant_gender'],
                    'participant_status': row_dict['participant_status'],
                    'participant_type': row_dict['participant_type']
                }
                geo_info = {
                    'latitude': float(row_dict['latitude']),
                    'longitude': float(row_dict['longitude'])
                }
                gun_info = {
                    'gun_stolen': row_dict['gun_stolen'],
                    'gun_type': row_dict['gun_type']
                }
                date_info = {
                    'date_fk': int(row_dict['date_fk'])
                }

                # add data into dimension tables
                partecipant_fk, gun_fk, date_fk, geo_fk = data_into_tables([partec_info, gun_info, geo_info, date_info],  dimensional_tables)
                
                # aggiungi incident info
                incident_id = int(row_dict['incident_id'])
                incident.add(incident_id)

                # calcola crime_gravity 
                crime_gravity = compute_gravity(age, type, status, partec_info)
                
                custody_id = int(row_dict['custody_id'])
                custody_record = [custody_id, partecipant_fk, gun_fk, geo_fk, date_fk,crime_gravity, incident_id] 
                custody[custody_id] = custody_record

In [ ]:
def read_xml_file(file_path):
    xml_data = []
    tree = ET.parse(file_path)
    root = tree.getroot()
    for row in root.findall('row'):
        data = {
            'date': row.find('date').text,
            'date_pk': row.find('date_pk').text
        }
        xml_data.append(data)
    return xml_data


In [ ]:
def enrich_date(date_table, xml_data):
    for row in xml_data:
        date_pk = int(row['date_pk'])
        date_info = [row['date'].split()[0], None, None, None, None, None]

        try:
            # estraggo  year, month, day 
            year, month, day = map(int, date_info[0].split('-'))
            date_info[3] = year
            date_info[2] = month
            date_info[1] = day

            # calcolo trimestre
            quarter = (month - 1) // 3 + 1
            date_info[4] = quarter

            # calcolo giorno della settimana
            datetime_obj = datetime.datetime(year, month, day)
            day_of_week_str = datetime_obj.strftime("%A")
            date_info[5] = day_of_week_str
        except:
            pass

        date_table[date_pk] = date_info


In [287]:
def enrich_geography(geography):
    def country_to_continent(country_name):
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
        return country_continent_name

    coords = [k for k in geography.keys()]
    results = (rg.search(coords))
    coordinates_to_results = dict(zip(coords, results))

    for coord, geo_id in geography.items():
        if coord in coordinates_to_results:
            result = coordinates_to_results[coord]
            continent = country_to_continent(result['country'])
            geography[coord] = [geo_id, result['city'], result['country'], continent] 


In [288]:
# inizializzazione delle strutture che rappresenteranno le tabelle (sia fact che dimensions)
partecipant = {}
geography = {}
gun = {}
datesprova = {}
incident = set()
custody = {}

dimensional_tables = {
    'partecipant': (partecipant, True) ,  # True if we need to generate the ID, False otherwise
    'gun': (gun, True) ,
    'geography': (geography, True) ,
    'datesprova': (datesprova, False)
}

file_partecipant_age = 'dict_partecipant_age.json'
file_partecipant_status = 'dict_partecipant_status.json'
file_partecipant_type = 'dict_partecipant_type.json'
file_police = 'Police.csv'
file_xml = 'dates.xml'

with open(file_partecipant_age, 'r') as F1:
    age = json.load(F1)    
with open(file_partecipant_type, 'r') as F2:
    type = json.load(F2)
with open(file_partecipant_status, 'r') as F3:
    status = json.load(F3)


process_police(file_police, dimensional_tables, incident, custody, age, type, status)
enrich_date(datesprova, read_xml_file(file_xml))
enrich_geography(geography)